<a href="https://colab.research.google.com/github/Delta9529/Car_Insurance_Cold_Calls/blob/main/Car_Insurance_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Car Insurance




Importing Libraries and Dataset

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dphi/Car_Insurance/Train_data (1).csv') #importing data

In [ ]:
data.head()

,age,job_type,marital_status,education_level,default_or_not,balance_amt,household_insurance,car_loan,communication,last_contact_day,last_contact_month,no_of_contacts,days_passed,prev_attempts,Outcome,call_start,call_end,car_insurance
0,47.000000,blue-collar,married,primary,0,1112.000000,1,0,cellular,13,may,1,-1,0,NaN,10:22:20,10:24:29,0
1,46.031442,management,married,tertiary,0,1.312372,0,1,cellular,15,may,3,219,1,other,09:06:30,13:59:48,1
2,36.000000,services,single,NaN,0,281.000000,1,0,NaN,16,may,1,-1,0,NaN,13:18:41,13:27:16,1
3,32.000000,technician,single,secondary,0,540.000000,0,1,cellular,29,jan,1,-1,0,NaN,10:09:18,10:21:48,1
4,30.000000,admin.,single,secondary,0,315.000000,1,0,cellular,14,may,3,346,4,failure,12:12:30,12:27:14,1


In [ ]:
data.dtypes

age                    float64
job_type                object
marital_status          object
education_level         object
default_or_not           int64
balance_amt            float64
household_insurance      int64
car_loan                 int64
communication           object
last_contact_day         int64
last_contact_month      object
no_of_contacts           int64
days_passed              int64
prev_attempts            int64
Outcome                 object
call_start              object
call_end                object
car_insurance            int64
dtype: object

# Data Preparation


In [ ]:
#converting age from float to int
data["age"] = data['age'].astype(int)

In [ ]:
data.isnull().sum() #Checking missing values (NOTE - 'days_passed' has missing values given as -1)

age                       0
job_type                 11
marital_status            0
education_level         113
default_or_not            0
balance_amt               0
household_insurance       0
car_loan                  0
communication           543
last_contact_day          0
last_contact_month        0
no_of_contacts            0
days_passed               0
prev_attempts             0
Outcome                2351
call_start                0
call_end                  0
car_insurance             0
dtype: int64

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

In [ ]:
data['job_type'] = imputer.fit_transform(data[['job_type']])     #Imputing missing categorical value
data['education_level'] = imputer.fit_transform(data[['education_level']])

In [ ]:
data.isnull().sum()

age                       0
job_type                  0
marital_status            0
education_level           0
default_or_not            0
balance_amt               0
household_insurance       0
car_loan                  0
communication           543
last_contact_day          0
last_contact_month        0
no_of_contacts            0
days_passed               0
prev_attempts             0
Outcome                2351
call_start                0
call_end                  0
car_insurance             0
dtype: int64

In [ ]:
data = data.drop(['communication','Outcome','last_contact_day','last_contact_month'],axis = 1) #communication & outcome having lots of missing values

In [ ]:
data['CallDuration'] = (pd.to_datetime(data['call_end']) - pd.to_datetime(data['call_start'])).apply(lambda x: x.seconds) #Getting an integer value for call duration
data = data.drop(['call_start', 'call_end'], axis=1)

In [ ]:
data.head()

,age,job_type,marital_status,education_level,default_or_not,balance_amt,household_insurance,car_loan,no_of_contacts,days_passed,prev_attempts,car_insurance,CallDuration
0,47,blue-collar,married,primary,0,1112.000000,1,0,1,-1,0,0,129
1,46,management,married,tertiary,0,1.312372,0,1,3,219,1,1,17598
2,36,services,single,secondary,0,281.000000,1,0,1,-1,0,1,515
3,32,technician,single,secondary,0,540.000000,0,1,1,-1,0,1,750
4,30,admin.,single,secondary,0,315.000000,1,0,3,346,4,1,884


In [ ]:
data['days_passed'] = data['days_passed'].replace(-1, np.nan) #Replacing missing values(-1) in day_passed column but dropping it because it has more than 50% of missing data

In [ ]:
data = data.drop('days_passed',axis = 1)

Encoding categorical columns

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
data['job_type'] = labelencoder.fit_transform(data['job_type'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
le_1 = LabelEncoder()
data['marital_status']=le_1.fit_transform(data['marital_status'])
le_2 = LabelEncoder()
data['education_level']=le_2.fit_transform(data['education_level'])

In [ ]:
data.head()

,age,job_type,marital_status,education_level,default_or_not,balance_amt,household_insurance,car_loan,no_of_contacts,prev_attempts,car_insurance,CallDuration
0,47,1,1,0,0,1112.000000,1,0,1,0,0,129
1,46,4,1,2,0,1.312372,0,1,3,1,1,17598
2,36,7,2,1,0,281.000000,1,0,1,0,1,515
3,32,9,2,1,0,540.000000,0,1,1,0,1,750
4,30,0,2,1,0,315.000000,1,0,3,4,1,884


Train and validation step

In [ ]:
x = data.drop('car_insurance',axis = 1)
y = data.car_insurance
print(x.shape)
print(y.shape)

(3102, 11)
(3102,)


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val = train_test_split(x,y,test_size=0.25,random_state=0)

# Fitting classifier models

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier0 = LogisticRegression(random_state=0)
classifier0.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_pred = classifier0.predict(x_val)

In [ ]:
from sklearn.metrics import accuracy_score
acc= accuracy_score(y_val,y_pred)
acc

0.8079896907216495

## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
classifier1 = DecisionTreeClassifier(criterion = 'entropy',random_state =0)
classifier1.fit(x_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [ ]:
y_pred_dt = classifier1.predict(x_val)

In [ ]:
from sklearn.metrics import accuracy_score
acc= accuracy_score(y_val,y_pred_dt)
acc

0.7938144329896907

## RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier2 = RandomForestClassifier(n_estimators = 1000,max_depth = 5,criterion ='entropy',random_state = 0)
classifier2.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
y_pred_rf = classifier2.predict(x_val)

In [ ]:
from sklearn.metrics import accuracy_score
acc= accuracy_score(y_val,y_pred_rf)
acc

0.8402061855670103

# Importing and Preparing Test Data

In [ ]:
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dphi/Car_Insurance/Test_data (1).csv')

In [ ]:
test_data.head()

,age,job_type,marital_status,education_level,default_or_not,balance_amt,household_insurance,car_loan,communication,last_contact_day,last_contact_month,no_of_contacts,days_passed,prev_attempts,Outcome,call_start,call_end
0,65.000000,retired,married,secondary,0,20806.0,0,0,telephone,8,apr,2,-1,0,NaN,12:22:13,12:30:56
1,36.000000,management,married,tertiary,0,900.0,1,0,NaN,15,may,2,-1,0,NaN,17:59:45,18:07:40
2,37.000000,management,married,tertiary,0,6771.0,0,0,cellular,7,sep,1,186,1,success,14:05:19,14:10:56
3,30.065066,management,single,tertiary,0,0.0,0,0,cellular,14,may,2,-1,0,NaN,17:33:27,09:47:54
4,41.000000,management,married,tertiary,0,328.0,1,0,cellular,29,jul,12,-1,0,NaN,14:40:26,14:41:31


In [ ]:
test_data["age"] = test_data['age'].astype(int)

In [ ]:
test_data.isnull().sum()

age                      0
job_type                 5
marital_status           0
education_level         34
default_or_not           0
balance_amt              0
household_insurance      0
car_loan                 0
communication          165
last_contact_day         0
last_contact_month       0
no_of_contacts           0
days_passed              0
prev_attempts            0
Outcome                695
call_start               0
call_end                 0
dtype: int64

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

In [ ]:
test_data['job_type'] = imputer.fit_transform(test_data[['job_type']])
test_data['education_level'] = imputer.fit_transform(test_data[['education_level']])

In [ ]:
test_data.isnull().sum()

age                      0
job_type                 0
marital_status           0
education_level          0
default_or_not           0
balance_amt              0
household_insurance      0
car_loan                 0
communication          165
last_contact_day         0
last_contact_month       0
no_of_contacts           0
days_passed              0
prev_attempts            0
Outcome                695
call_start               0
call_end                 0
dtype: int64

In [ ]:
test_data = test_data.drop(['communication','Outcome','last_contact_day','last_contact_month','days_passed'],axis = 1)

In [ ]:
test_data['CallDuration'] = (pd.to_datetime(test_data['call_end']) - pd.to_datetime(test_data['call_start'])).apply(lambda x: x.seconds)
test_data = test_data.drop(['call_start', 'call_end'], axis=1)

In [ ]:
test_data.head()

,age,job_type,marital_status,education_level,default_or_not,balance_amt,household_insurance,car_loan,no_of_contacts,prev_attempts,CallDuration
0,65,retired,married,secondary,0,20806.0,0,0,2,0,523
1,36,management,married,tertiary,0,900.0,1,0,2,0,475
2,37,management,married,tertiary,0,6771.0,0,0,1,1,337
3,30,management,single,tertiary,0,0.0,0,0,2,0,58467
4,41,management,married,tertiary,0,328.0,1,0,12,0,65


In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
test_data['job_type'] = labelencoder.fit_transform(test_data['job_type'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
le_1 = LabelEncoder()
test_data['marital_status']=le_1.fit_transform(test_data['marital_status'])
le_2 = LabelEncoder()
test_data['education_level']=le_2.fit_transform(test_data['education_level'])

In [ ]:
test_data.head()

,age,job_type,marital_status,education_level,default_or_not,balance_amt,household_insurance,car_loan,no_of_contacts,prev_attempts,CallDuration
0,65,5,1,1,0,20806.0,0,0,2,0,523
1,36,4,1,2,0,900.0,1,0,2,0,475
2,37,4,1,2,0,6771.0,0,0,1,1,337
3,30,4,2,2,0,0.0,0,0,2,0,58467
4,41,4,1,2,0,328.0,1,0,12,0,65


## Prediction

In [ ]:
target = classifier2.predict(test_data)

Downloading the target column into csv file

In [ ]:
res = pd.DataFrame(target)
res.index = test_data.index
res.columns = ["prediction"]

from google.colab import files
res.to_csv('target_car_new.csv')         
files.download('target_car_new.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>